In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno 
import re
import folium

In [12]:
!pip install folium 

     |████████████████████████████████| 93 kB 783 kB/s eta 0:00:011


In [14]:
df=pd.read_csv("sales10000.csv") 

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (53,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
sales = df.groupby('Ciudad').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
sales.head(10)

,Ciudad,count
58,cucuta,2154
24,bogota,1415
26,bucaramanga,843
136,pereira,835
17,barranquilla,538
33,cali,489
117,medellin,445
37,cartagena,216
126,neiva,188
190,soacha,136


In [16]:
from folium.plugins import HeatMap
def generateBaseMap(default_location=[4.710989,-74.072092], default_zoom_start=5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [17]:
basemap  = generateBaseMap()
basemap

In [18]:
base_map = generateBaseMap()
df['sales'] = 1

HeatMap(data=df[['lat', 'lon', 'sales']].groupby(['lat', 'lon']).sum().reset_index().values.tolist(), radius=15, max_zoom=13).add_to(base_map)
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))

In [22]:
df_map= df[['Ciudad','lat', 'lon', 'sales']].groupby(['Ciudad','lat', 'lon']).sum().reset_index()
df_map.head()

,Ciudad,lat,lon,sales
0,acacias,41.667045,-91.541691,8
1,acopi yumbo,3.497459,-76.508356,1
2,aguachica,8.309746,-73.614487,37
3,aguadas,5.613574,-75.457514,1
4,agustn codazzi,10.037216,-73.228411,1


In [24]:
df_map2= df[['Ciudad','lat', 'lon', 'sales','DATE']].groupby(['Ciudad','lat', 'lon','DATE']).sum().reset_index()
df_map2.head()

,Ciudad,lat,lon,DATE,sales
0,acacias,41.667045,-91.541691,2014-02,1
1,acacias,41.667045,-91.541691,2015-04,1
2,acacias,41.667045,-91.541691,2015-07,1
3,acacias,41.667045,-91.541691,2015-12,1
4,acacias,41.667045,-91.541691,2016-07,1


In [26]:
base_map = folium.Map(location=[4.710989,-74.072092], zoom_start=5)

df_map.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lon"]],popup= row['Ciudad'],radius=1,color='white',opacity=0.1).add_to(base_map), axis=1)

HeatMap(data=df[['lat', 'lon', 'sales']].groupby(['lat', 'lon']).sum().reset_index().values.tolist(), radius=15,     max_zoom=13).add_to(base_map)

In [27]:
base_map

In [29]:
df_map['Percentile']=df_map.sales.rank(pct=True)

df_map.head()

,Ciudad,lat,lon,sales,Percentile
0,acacias,41.667045,-91.541691,8,0.652466
1,acopi yumbo,3.497459,-76.508356,1,0.163677
2,aguachica,8.309746,-73.614487,37,0.865471
3,aguadas,5.613574,-75.457514,1,0.163677
4,agustn codazzi,10.037216,-73.228411,1,0.163677


In [33]:
lat = list(df_map["lat"])
lon = list(df_map["lon"])
Percentile = list(df_map["Percentile"])
sales = list(df_map["sales"])
ciudad = list(df_map["Ciudad"])
def rad_size(number):
    if number < 10:
        return 2,'#fef0d9'
    elif 10 <= number and number< 100:
        return 5,'#fec44f'
    elif 100 <= number and number< 500:
        return 10,'#ffff33'
    elif 500 <= number and number< 1000:
        return 15,'#ff7f00'
    else:
        return 20,'#de2d26'

In [35]:
# df.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lon"]],popup= row['Ciudad']
#                                         +'\n Number of sales:'+ str(row['sales'] ,radius=no.log(row["sales"]),color=color_producer(row["Percentile"]),opacity=0.7,
#                                          fill=True,fill_opacity=0.5,fill_color=color_producer(row["Percentile"])).add_to(base_map), axis=1))

In [36]:
import branca
import branca.colormap as cmp
linear = cmp.LinearColormap(
    ['#fef0d9', '#fec44f', '#ffff33','#ff7f00','#de2d26'],
    index=[0,100, 500, 1000, 3000],
    vmin=1, vmax=3000,
    caption='Number of Sales' #Caption for Color scale or Legend
)


base_map = folium.Map(location=[4.710989,-74.072092], zoom_start=5)
fg = folium.FeatureGroup(name="My Map")
for lt, ln, per,sale,ciu in zip(lat, lon,Percentile, sales,ciudad):
    fg.add_child(folium.CircleMarker(location=[lt, ln], radius =rad_size(sale)[0] , popup=str(ciu)+" \n number of sales:"+str(sale),
    fill_color=rad_size(sale)[1], fill=True ,fill_opacity=0.7,color='Black',opacity = 0.4))
    

base_map.add_child(fg)
linear.add_to(base_map)
base_map

In [46]:
# df_map2.to_csv('sales_grouped_date.csv')

In [47]:
df_map2.head()

,Ciudad,lat,lon,DATE,sales
0,acacias,41.667045,-91.541691,2014-02,1
1,acacias,41.667045,-91.541691,2015-04,1
2,acacias,41.667045,-91.541691,2015-07,1
3,acacias,41.667045,-91.541691,2015-12,1
4,acacias,41.667045,-91.541691,2016-07,1


In [60]:
import branca
import branca.colormap as cmp
linear = cmp.LinearColormap(
    ['#fef0d9', '#fec44f', '#ffff33','#ff7f00','#de2d26'],
    index=[0,100, 500, 1000, 3000],
    vmin=1, vmax=3000,
    caption='Number of Sales' #Caption for Color scale or Legend
)


base_map = folium.Map(location=[4.710989,-74.072092], zoom_start=5)
fg = folium.FeatureGroup(name="My Map")
for lt, ln, per,sale,ciu in zip(lat, lon,Percentile, sales,ciudad):
    fg.add_child(folium.CircleMarker(location=[lt, ln], radius =rad_size(sale)[0] , popup=str(ciu)+" \n number of sales:"+str(sale),
    fill_color=rad_size(sale)[1], fill=True ,fill_opacity=0.7,color='Black',opacity = 0.4))
    

base_map.add_child(fg)
linear.add_to(base_map)

base_map

In [72]:

df_date=pd.read_csv("sales_grouped_date.csv") 
# df_map2['DATE'] = pd.to_datetime(df_map2['DATE']).dt.to_period('Y')
# df_map2[df_map2['DATE'] == '2014']
# df_date = df_date[df_date['date'].dt.strftime('%Y') == '2014']
df_date['year'] = pd.to_datetime(df_date['DATE']).dt.strftime('%Y')

In [73]:
df_date.head()

,Unnamed: 0,Ciudad,lat,lon,DATE,sales,year
0,0,acacias,41.667045,-91.541691,2014-02,1,2014
1,1,acacias,41.667045,-91.541691,2015-04,1,2015
2,2,acacias,41.667045,-91.541691,2015-07,1,2015
3,3,acacias,41.667045,-91.541691,2015-12,1,2015
4,4,acacias,41.667045,-91.541691,2016-07,1,2016


In [93]:
df_year = df_date[df_date['year'] == '2010']


In [94]:
df_year_map = df_year[['Ciudad','lat', 'lon', 'sales']].groupby(['Ciudad','lat', 'lon']).sum().reset_index()

In [95]:
lat = list(df_year_map["lat"])
lon = list(df_year_map["lon"])
sales = list(df_year_map["sales"])
ciudad = list(df_year_map["Ciudad"])


import branca
import branca.colormap as cmp
linear = cmp.LinearColormap(
    ['#fef0d9', '#fec44f', '#ffff33','#ff7f00','#de2d26'],
    index=[0,100, 500, 1000, 3000],
    vmin=1, vmax=3000,
    caption='Number of Sales' #Caption for Color scale or Legend
)


base_map = folium.Map(location=[4.710989,-74.072092], zoom_start=5)
fg = folium.FeatureGroup(name="My Map")
for lt, ln, per,sale,ciu in zip(lat, lon,Percentile, sales,ciudad):
    fg.add_child(folium.CircleMarker(location=[lt, ln], radius =rad_size(sale)[0] , popup=str(ciu)+" \n number of sales:"+str(sale),
    fill_color=rad_size(sale)[1], fill=True ,fill_opacity=0.7,color='Black',opacity = 0.4))
    

base_map.add_child(fg)
linear.add_to(base_map)
base_map

In [90]:
df_date2 = df_date.sort_values('year',ascending=False)

In [92]:
df_date2.tail()

,Unnamed: 0,Ciudad,lat,lon,DATE,sales,year
2822,2822,tulu,43.492981,-70.452839,2010-10,1,2010
1659,1659,medellin,6.247638,-75.565815,2010-01,1,2010
1660,1660,medellin,6.247638,-75.565815,2010-02,2,2010
1661,1661,medellin,6.247638,-75.565815,2010-09,3,2010
2095,2095,pereira,4.808717,-75.690601,2010-09,5,2010


In [98]:
df_year_map.head()

,Ciudad,lat,lon,sales
0,apartado,7.882761,-76.624692,1
1,arauca,6.547306,-71.002231,3
2,armenia,40.069099,45.038189,2
3,barrancabermeja,7.061709,-73.851932,5
4,barranquilla,11.004107,-74.806981,40


In [99]:
Total = df_year_map['sales'].sum()

In [100]:
Total

504